#  🏏 T20 Cricket Score Predictor


### Overview  
A machine learning model that predicts the final T20 cricket score in real-time based on current match conditions and recent performance trends.  

### How It Works  
The model analyzes the current match state and uses **XGBoost** (Gradient Boosting) to predict the final score by considering:  
- Current score, wickets, and balls remaining  
- Run rate trends (overall and last 5 overs)  
- Historical performance of teams at specific venues  
- Recent momentum (wickets and runs in last 5 overs)  

### Key Features  
- **Real-time Predictions**: Get score forecasts at any point during an innings  
- **Context-Aware**: Accounts for venue, teams, and match situation  
- **Momentum Analysis**: Weights recent performance (last 5 overs) heavily  
- **High Accuracy**: Trained on extensive ball-by-ball historical data

## 📊🔍Data Extraction

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("t20i_info.csv")

In [3]:
df.shape

(63888, 9)

In [4]:
df.head()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground


In [5]:
df.isnull().sum()

Unnamed: 0             0
match_id               0
batting_team           0
bowling_team           0
ball                   0
runs                   0
player_dismissed       0
city                8548
venue                  0
dtype: int64

In [6]:
df.drop('city', axis=1, inplace=True)

In [7]:
df.isnull().sum()

Unnamed: 0          0
match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
venue               0
dtype: int64

In [8]:
print(df['venue'].value_counts())
eligible_venues = df['venue'].value_counts()[df['venue'].value_counts() >= 30].index.tolist()
df = df[df['venue'].isin(eligible_venues)]

venue
Shere Bangla National Stadium          3420
R Premadasa Stadium                    2983
Dubai International Cricket Stadium    2969
New Wanderers Stadium                  2819
Eden Park                              2532
                                       ... 
Senwes Park                             122
Sardar Patel Stadium, Motera            121
Hagley Oval                             121
Subrata Roy Sahara Stadium              121
Carrara Oval                             64
Name: count, Length: 94, dtype: int64


In [9]:
df.shape

(63888, 8)

In [10]:
df.head()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,venue
0,0,2,Australia,Sri Lanka,0.1,0,0,Melbourne Cricket Ground
1,1,2,Australia,Sri Lanka,0.2,0,0,Melbourne Cricket Ground
2,2,2,Australia,Sri Lanka,0.3,1,0,Melbourne Cricket Ground
3,3,2,Australia,Sri Lanka,0.4,2,0,Melbourne Cricket Ground
4,4,2,Australia,Sri Lanka,0.5,0,0,Melbourne Cricket Ground


In [11]:
df['current_score'] = df.groupby('match_id')['runs'].cumsum()

In [12]:
df.head()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,venue,current_score
0,0,2,Australia,Sri Lanka,0.1,0,0,Melbourne Cricket Ground,0
1,1,2,Australia,Sri Lanka,0.2,0,0,Melbourne Cricket Ground,0
2,2,2,Australia,Sri Lanka,0.3,1,0,Melbourne Cricket Ground,1
3,3,2,Australia,Sri Lanka,0.4,2,0,Melbourne Cricket Ground,3
4,4,2,Australia,Sri Lanka,0.5,0,0,Melbourne Cricket Ground,3


In [13]:
df['over'] = df['ball'].apply(lambda x : str(x).split(".")[0])
df['ball_no'] = df['ball'].apply(lambda x : str(x).split(".")[1])

In [14]:
df.head()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,venue,current_score,over,ball_no
0,0,2,Australia,Sri Lanka,0.1,0,0,Melbourne Cricket Ground,0,0,1
1,1,2,Australia,Sri Lanka,0.2,0,0,Melbourne Cricket Ground,0,0,2
2,2,2,Australia,Sri Lanka,0.3,1,0,Melbourne Cricket Ground,1,0,3
3,3,2,Australia,Sri Lanka,0.4,2,0,Melbourne Cricket Ground,3,0,4
4,4,2,Australia,Sri Lanka,0.5,0,0,Melbourne Cricket Ground,3,0,5


In [15]:
df['ball_bowled'] = (df['over'].astype(int)*6 + df['ball_no'].astype(int))

In [16]:
df['balls_left'] = 120 - df['ball_bowled']

In [17]:
df.tail()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,venue,current_score,over,ball_no,ball_bowled,balls_left
63883,121,964,Sri Lanka,Australia,19.3,1,0,R Premadasa Stadium,125,19,3,117,3
63884,122,964,Sri Lanka,Australia,19.4,0,0,R Premadasa Stadium,125,19,4,118,2
63885,123,964,Sri Lanka,Australia,19.5,0,DM de Silva,R Premadasa Stadium,125,19,5,119,1
63886,124,964,Sri Lanka,Australia,19.6,2,0,R Premadasa Stadium,127,19,6,120,0
63887,125,964,Sri Lanka,Australia,19.7,1,0,R Premadasa Stadium,128,19,7,121,-1


In [18]:
df['balls_left'] = df['balls_left'].apply(lambda x:0 if x<0 else x)

In [19]:
df.head()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,venue,current_score,over,ball_no,ball_bowled,balls_left
0,0,2,Australia,Sri Lanka,0.1,0,0,Melbourne Cricket Ground,0,0,1,1,119
1,1,2,Australia,Sri Lanka,0.2,0,0,Melbourne Cricket Ground,0,0,2,2,118
2,2,2,Australia,Sri Lanka,0.3,1,0,Melbourne Cricket Ground,1,0,3,3,117
3,3,2,Australia,Sri Lanka,0.4,2,0,Melbourne Cricket Ground,3,0,4,4,116
4,4,2,Australia,Sri Lanka,0.5,0,0,Melbourne Cricket Ground,3,0,5,5,115


In [20]:
df['player_dismissed'] = df['player_dismissed'].apply(lambda x:1 if x != '0' else '0')

In [21]:
df['player_dismissed'] = df['player_dismissed'].astype(int)

In [22]:
df.head()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,venue,current_score,over,ball_no,ball_bowled,balls_left
0,0,2,Australia,Sri Lanka,0.1,0,0,Melbourne Cricket Ground,0,0,1,1,119
1,1,2,Australia,Sri Lanka,0.2,0,0,Melbourne Cricket Ground,0,0,2,2,118
2,2,2,Australia,Sri Lanka,0.3,1,0,Melbourne Cricket Ground,1,0,3,3,117
3,3,2,Australia,Sri Lanka,0.4,2,0,Melbourne Cricket Ground,3,0,4,4,116
4,4,2,Australia,Sri Lanka,0.5,0,0,Melbourne Cricket Ground,3,0,5,5,115


In [23]:
df['player_dismissed'] = df.groupby('match_id')['player_dismissed'].cumsum()

In [24]:
df['wicket_left'] = 10 - df['player_dismissed']

In [25]:
df['current_run_rate'] = (df['current_score']*6 / df['ball_bowled'])

In [26]:
df.tail()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,venue,current_score,over,ball_no,ball_bowled,balls_left,wicket_left,current_run_rate
63883,121,964,Sri Lanka,Australia,19.3,1,8,R Premadasa Stadium,125,19,3,117,3,2,6.410256
63884,122,964,Sri Lanka,Australia,19.4,0,8,R Premadasa Stadium,125,19,4,118,2,2,6.355932
63885,123,964,Sri Lanka,Australia,19.5,0,9,R Premadasa Stadium,125,19,5,119,1,1,6.302521
63886,124,964,Sri Lanka,Australia,19.6,2,9,R Premadasa Stadium,127,19,6,120,0,1,6.350000
63887,125,964,Sri Lanka,Australia,19.7,1,9,R Premadasa Stadium,128,19,7,121,0,1,6.347107


In [54]:
groups = df.groupby('match_id')

last_five = []

for match_id in df['match_id'].unique():
    # Calculate rolling sum of PREVIOUS 30 balls (not including current)
    rolling_sum = groups.get_group(match_id)['runs'].rolling(window=30).sum().shift(1).fillna(0)
    last_five.extend(rolling_sum.values.tolist())

df['last_five'] = last_five

In [58]:
final_df = df.groupby('match_id').sum()['runs'].reset_index().merge(df,on="match_id")

In [59]:
final_df

,match_id,runs_x,Unnamed: 0,batting_team,bowling_team,ball,runs_y,player_dismissed,venue,current_score,over,ball_no,ball_bowled,balls_left,wicket_left,current_run_rate,last_five
0,2,168,0,Australia,Sri Lanka,0.1,0,0,Melbourne Cricket Ground,0,0,1,1,119,10,0.000000,0.0
1,2,168,1,Australia,Sri Lanka,0.2,0,0,Melbourne Cricket Ground,0,0,2,2,118,10,0.000000,0.0
2,2,168,2,Australia,Sri Lanka,0.3,1,0,Melbourne Cricket Ground,1,0,3,3,117,10,2.000000,0.0
3,2,168,3,Australia,Sri Lanka,0.4,2,0,Melbourne Cricket Ground,3,0,4,4,116,10,4.500000,0.0
4,2,168,4,Australia,Sri Lanka,0.5,0,0,Melbourne Cricket Ground,3,0,5,5,115,10,3.600000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,128,121,Sri Lanka,Australia,19.3,1,8,R Premadasa Stadium,125,19,3,117,3,2,6.410256,35.0
63884,964,128,122,Sri Lanka,Australia,19.4,0,8,R Premadasa Stadium,125,19,4,118,2,2,6.355932,32.0
63885,964,128,123,Sri Lanka,Australia,19.5,0,9,R Premadasa Stadium,125,19,5,119,1,1,6.302521,32.0
63886,964,128,124,Sri Lanka,Australia,19.6,2,9,R Premadasa Stadium,127,19,6,120,0,1,6.350000,32.0


In [60]:
final_df.columns

Index(['match_id', 'runs_x', 'Unnamed: 0', 'batting_team', 'bowling_team',
       'ball', 'runs_y', 'player_dismissed', 'venue', 'current_score', 'over',
       'ball_no', 'ball_bowled', 'balls_left', 'wicket_left',
       'current_run_rate', 'last_five'],
      dtype='object')

In [61]:
final_df = final_df[['batting_team', 'bowling_team','venue','balls_left', 'current_score',  'wicket_left',
       'current_run_rate', 'last_five','runs_x']]

In [32]:
final_df.dropna(inplace=True)

/tmp/ipykernel_9916/2709626079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.dropna(inplace=True)


In [62]:
final_df.isnull().sum()

batting_team        0
bowling_team        0
venue               0
balls_left          0
current_score       0
wicket_left         0
current_run_rate    0
last_five           0
runs_x              0
dtype: int64

In [63]:
final_df.shape

(63888, 9)

## 🧠📊Model Training

In [64]:
final_df.head()

,batting_team,bowling_team,venue,balls_left,current_score,wicket_left,current_run_rate,last_five,runs_x
0,Australia,Sri Lanka,Melbourne Cricket Ground,119,0,10,0.0,0.0,168
1,Australia,Sri Lanka,Melbourne Cricket Ground,118,0,10,0.0,0.0,168
2,Australia,Sri Lanka,Melbourne Cricket Ground,117,1,10,2.0,0.0,168
3,Australia,Sri Lanka,Melbourne Cricket Ground,116,3,10,4.5,0.0,168
4,Australia,Sri Lanka,Melbourne Cricket Ground,115,3,10,3.6,0.0,168


In [65]:
X = final_df.drop(columns=['runs_x'])
y = final_df['runs_x']

In [66]:
X

,batting_team,bowling_team,venue,balls_left,current_score,wicket_left,current_run_rate,last_five
0,Australia,Sri Lanka,Melbourne Cricket Ground,119,0,10,0.000000,0.0
1,Australia,Sri Lanka,Melbourne Cricket Ground,118,0,10,0.000000,0.0
2,Australia,Sri Lanka,Melbourne Cricket Ground,117,1,10,2.000000,0.0
3,Australia,Sri Lanka,Melbourne Cricket Ground,116,3,10,4.500000,0.0
4,Australia,Sri Lanka,Melbourne Cricket Ground,115,3,10,3.600000,0.0
...,...,...,...,...,...,...,...,...
63883,Sri Lanka,Australia,R Premadasa Stadium,3,125,2,6.410256,35.0
63884,Sri Lanka,Australia,R Premadasa Stadium,2,125,2,6.355932,32.0
63885,Sri Lanka,Australia,R Premadasa Stadium,1,125,1,6.302521,32.0
63886,Sri Lanka,Australia,R Premadasa Stadium,0,127,1,6.350000,32.0


In [67]:
from sklearn.model_selection import train_test_split

In [68]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [69]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

In [70]:
transformer = ColumnTransformer([
    ('trf', OneHotEncoder(sparse_output=False, drop='first'), ['batting_team', 'bowling_team', 'venue'])
], remainder='passthrough')

In [71]:
pipe = Pipeline(steps=[
    ('step1',transformer),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state = 1)),  
])

In [72]:
pipe.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('step1', ...), ('step2', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('trf', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sp

In [73]:
y_pred = pipe.predict(X_test)

In [74]:
r2_score(y_test,y_pred)

0.9693306684494019

In [75]:
mean_absolute_error(y_test,y_pred)

2.19028902053833

In [76]:
pipe.score(X_test,y_test)

0.9693306684494019

In [77]:
import pickle
pickle.dump(pipe, open('pipe.pkl','wb'))